<a href="https://colab.research.google.com/github/Raphael181/Raphael181/blob/main/TensorFlow_Classification_of_Flowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x

from __future__ import absolute_import, division , print_function, unicode_literals 

import tensorflow as tf

import pandas as pd

In [ ]:
CSV_COLUMN_NAMES = ['Sepallength', 'SepalWidth' , 'PetalLength', 'PetalWidth', 'Species']
SPECIES  = ['Setosa', 'Versicolor', 'Virginica']


In [ ]:
train_path = tf.keras.utils.get_file('iris_training.csv' , 'https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv')
test_path = tf.keras.utils.get_file('iris_test.csv' , 'https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv')
#We use keras, a model inside of tensorflow to grab our datasets and store them
train = pd.read_csv(train_path, names = CSV_COLUMN_NAMES  , header = 0)
test = pd.read_csv(test_path, names = CSV_COLUMN_NAMES , header=0)

8192/573 [============================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


In [ ]:
train.head()
train_y = train.pop("Species")
test_y  = test.pop("Species")

In [ ]:

train.head()

,Sepallength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [ ]:
train_y.head()

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

In [ ]:
#input function
def input_fn(features, labels, training = True , batch_size = 256):
  #convert the inputs to a dataset.
  datasets = tf.data.Dataset.from_tensor_slices((dict(features) , labels ))

  #shuffle and repeat when in training mode
  if training:
    datasets = datasets.shuffle(1000).repeat()

  return datasets.batch(batch_size)

In [ ]:
feature_columns = []

for key in train.keys():
  feature_columns.append(tf.feature_column.numeric_column(key = key))

print(feature_columns)

[NumericColumn(key='Sepallength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [ ]:
#Building the Model
#Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns = feature_columns, 
    #Two hidden layesr of 30 and 10 nodes respectively
    hidden_units = [30,10],
    #The model must choose between 3 classes
    n_classes = 3
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpux3d4r8h', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
#Training the model:
classifier.train(
    input_fn = lambda: input_fn(train , train_y , training = True),
    steps =5000
)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpux3d4r8h/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.2328279, step = 0
INFO:tensorflow:global_step/sec: 354.34
INFO:tensorflow:loss = 0.9344884, step = 100 (0.285 sec)
INFO:tensorflow:global_step/sec: 477.022
INFO:tensorflow:loss = 0.8794469, step = 200 (0.210 sec)
INFO:tensorflow:global_step/sec:

In [ ]:
eval_result = classifier.evaluate(input_fn = lambda: input_fn(test , test_y , training = False))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-04-18T00:09:22Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpux3d4r8h/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.28305s
INFO:tensorflow:Finished evaluation at 2021-04-18-00:09:22
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.6666667, average_loss = 0.535065, global_step = 5000, loss = 0.535065
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmpux3d4r8h/model.ckpt-5000

Test set accuracy: 0.667



In [ ]:
#the below function allows the user to input some numbers, and spit out the predicted class based on the numbers you input
def input_fn(features, batch_size=256):
  #Converts the inputs to datasets without labels
  #when making a prediction, we don't know the labels 
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ["SepalLength", "SepalWidth", "PetalLength" , "PetalWidth"]
predict = {}

#print function that tells the user to enter a numeric value
print("Please enter numeric values as prompted:")

#outer for loop used to check for every item in the "features" list
for feature in features:
  #set a variable to true
  valid  = True
  while valid:
    val = input(feature + ":" )
    #if the val entered is not a digit, valid is false and keep prompting the user for a digital value
    if not val.isdigit(): valid = False
  
  #Add that value to our predict dictionary
  predict[feature] = {float(val)}

predictions = classifier.predict(input_fn =lambda: input_fn(predict))
for pre_dict in predictions:
  class_id = pre_dict["class_ids"][0]
  probability = pre_dict["probabilities"][class_id]

  print("Prediction is '{}' ({:.if}%)".format(
      SPECIES[class_id] , 100*probability
  ))

Please enter numeric values as prompted:
SepalLength:2.4
SepalWidth:2.6
PetalLength:6.5
PetalWidth:6.3


TypeError: ignored